In [ ]:
import pprint
from pathlib import Path

import tifffile
from careamics_portfolio import PortfolioManager
from matplotlib.pyplot import imshow, subplots

from careamics_restoration.engine import Engine
from careamics_restoration.metrics import psnr

### Import Dataset Portfolio

In [ ]:
# Explore portfolio
portfolio = PortfolioManager()
print(portfolio.denoising)

In [ ]:
# Download and unzip the files
root_path = Path("data")
files = portfolio.denoising.N2V_BSD68.download(root_path)
print(f"List of downloaded files: {files}")

In [ ]:
data_path = Path(root_path / "denoising-N2V_BSD68.unzip/BSD68_reproducibility_data")
train_path = data_path / "train"
val_path = data_path / "val"
test_path = data_path / "test" / "images"
gt_path = data_path / "test" / "gt"

train_path.mkdir(parents=True, exist_ok=True)
val_path.mkdir(parents=True, exist_ok=True)
test_path.mkdir(parents=True, exist_ok=True)
gt_path.mkdir(parents=True, exist_ok=True)

### Visualize training data

In [ ]:
train_image = tifffile.imread(next(iter(train_path.rglob("*.tiff"))))[0]
print(f"Train image shape: {train_image.shape}")
imshow(train_image, cmap="gray")

### Visualize validation data

In [ ]:
val_image = tifffile.imread(next(iter(val_path.rglob("*.tiff"))))[0]
print(f"Validation image shape: {val_image.shape}")
imshow(val_image, cmap="gray")

### Initialize the Engine

Engine contains the dataloading pipeline and the model training logic. We'll initialize the engine with the config file, but it can also be initialized from a pre-trained checkpoint.

Please take as look at the [documentation](https://careamics.github.io) to see the full list of parameters and configuration options

In [ ]:
engine = Engine(config_path="n2v_2D_BSD.yml")

### Visualize training configuration

In [ ]:
pprint.PrettyPrinter(indent=2).pprint(engine.cfg.model_dump(exclude_optionals=False))

### Run training 

We need to specify the paths to training and validation data

In [ ]:
train_stats, val_stats = engine.train(train_path=train_path, val_path=val_path)

### Visualize statistics

In [ ]:
import matplotlib.pyplot as plt

plt.plot([next(iter(d.values())) for d in train_stats], label="Train loss")
plt.plot([next(iter(d.values())) for d in val_stats], label="Validation loss")
plt.legend(loc="best")
plt.xlabel("Epoch")

### Visualize files to denoise

In [ ]:
test_img = tifffile.imread(test_path / "bsd68_gaussian25_1.tiff")
imshow(test_img, cmap="gray")
print(test_img.shape)

### Run prediction

We need to specify the path to the data we want to denoise

In [ ]:
preds = engine.predict(
    input=test_path, tile_shape=[64, 64], overlaps=[48, 48], axes="YX"
)

### Visualize results and compute metrics


In [ ]:
# Create a list of ground truth images

gts = [tifffile.imread(f) for f in sorted(gt_path.glob("*.tiff"))]

In [ ]:
# Plot single image

image_idx = 0
_, subplot = subplots(1, 2, figsize=(10, 10))

subplot[0].imshow(preds[image_idx], cmap="gray")
subplot[0].set_title("Prediction")
subplot[1].imshow(gts[image_idx], cmap="gray")
subplot[1].set_title("Ground truth")

In [ ]:
# Calculate PSNR for single image

psnr_single = psnr(gts[image_idx], preds[image_idx])
print(f"PSNR for image {image_idx}: {psnr_single}")

In [ ]:
psnr_total = 0

for pred, gt in zip(preds, gts):
    psnr_total += psnr(gt, pred)

print(f"PSNR total: {psnr_total / len(preds)}")

### Export to bioimage.io

In [ ]:
engine.save_as_bioimage(engine.cfg.experiment_name + "bioimage.zip")